In [30]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')
import logging
import datetime

import csv, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from dayu.hooks.oss_hook import OSSHook
from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            return datain
        tb_name, col_name = column.split('.')
        new_cols.append((column, col_name))
    datain = datain.rename(columns=dict(new_cols))
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
#     outdata = split_table_name(outdata)
    return outdata

## 计算时间差
def date_time_sub(startTime,endTime,date_format):
    try:
        startTime= datetime.datetime.strptime(startTime,date_format)
        endTime= datetime.datetime.strptime(endTime,date_format)
        return (endTime - startTime).days
    except:
        pass

    # 计算车辆当前保值率
def computer_with_license_month(tar):
    try:
        license_month = tar['license_month']
        if(license_month<=12):
            #tar['keep_value'] = tar['year_1']
            return tar['year_1']
        else:
            year = license_month//12
            #当前年保值率
            keep_max = tar["year_"+str(int(year))]
            #下一年的保值率
            keep_min = tar["year_"+str(int(year+1))]

            #相比于上一年，已经过了几个月
            mon = license_month-12*year
            tem = (keep_max-keep_min)/12

            #tar['keep_value'] = round(keep_max - tem*mon, 4)
            return round(keep_max - tem*mon, 4)
    except:
        return tar["year_16"]

class Logger:       
    def __init__(self, logName, logFile):
        self._logger = logging.getLogger(logName)
        handler = logging.FileHandler(logFile)
        formatter = logging.Formatter('%(asctime)s ********* %(message)s')
        handler.setFormatter(formatter)
        self._logger.addHandler(handler)
        self._logger.setLevel(logging.INFO)

    def log(self, msg):
        if self._logger is not None:
            self._logger.info(msg)


pd.set_option('display.max_columns', 500)
curr_dir = '/home/souche/qiongjiu/hgc/'

curr_date=str(datetime.datetime.now())[0:10]
hive_cli = HiveCliHook("warehouse_hive")

logger = Logger('model_service','./log/accurate_valuation_cyp_run_log.log')
logger.log("程序启动.............")
## 设定需要计算的年
computer_year = '2020'

[2021-01-07 15:37:03,740] {<ipython-input-30-28c4b78419c8>:84} INFO - 程序启动.............


In [31]:
sql_info ="""
select * 
from (
    SELECT
        car_id,
        brand_name,
        model_name,
        a_province_name,
        model_code,
        substr(publish_time,1,4)  pay_year,
        substr(license_time,1,4) license_year,
        second_price/10000 real_pay_amount,
        a_province_name,
        mileage,
        row_number() over(partition by model_code,mileage,license_time,a_province_name order by publish_time desc) num 
    FROM
        dl_outrs.dl_vehicle_data_scavenger_ipbl_t_reptile_car_dd
    WHERE
        ds = date_sub('"""+curr_date+"""',1) 
        and second_price/10000 >= 0.3    
        and second_price/10000 < 300 
        and mileage/10000 > 0     
        and mileage/10000 < 100 
        and site not in ('51','renrenche') 
        and substr(publish_time,0,4) in ('"""+computer_year+"""')
) t1
where t1.num = 1 
"""
dtype={'city_code':str}
dl_site_ts_order_clean = read_from_hive2('dl_site_ts_order_clean',sql_info,dtype)

db_columns = []
for col in dl_site_ts_order_clean.columns:
    if len(col.split('.')) > 1:
        db_columns.append(col.split('.')[1])

    else:
        db_columns.append(col)

dl_site_ts_order_clean.columns = db_columns 
logger.log("读取数据完成.............")



[2021-01-07 15:37:05,880] {hiveserver2:138} INFO - Using database default as default
[2021-01-07 15:37:06,018] {hive_server_hook:112} INFO - Running query: 
select * 
from (
    SELECT
        car_id,
        brand_name,
        model_name,
        a_province_name,
        model_code,
        substr(publish_time,1,4)  pay_year,
        substr(license_time,1,4) license_year,
        second_price/10000 real_pay_amount,
        a_province_name,
        mileage,
        row_number() over(partition by model_code,mileage,license_time,a_province_name order by publish_time desc) num 
    FROM
        dl_outrs.dl_vehicle_data_scavenger_ipbl_t_reptile_car_dd
    WHERE
        ds = date_sub('2021-01-07',1) 
        and second_price/10000 >= 0.3    
        and second_price/10000 < 300 
        and mileage/10000 > 0     
        and mileage/10000 < 100 
        and site not in ('51','renrenche') 
        and substr(publish_time,0,4) in ('2020')
) t1
where t1.num = 1 

[2021-01-07 15:38:07,638] {hi

[2021-01-07 15:38:44,556] {hive_server_hook:162} INFO - Written 880000 rows so far.
[2021-01-07 15:38:44,980] {hive_server_hook:162} INFO - Written 890000 rows so far.
[2021-01-07 15:38:45,380] {hive_server_hook:162} INFO - Written 900000 rows so far.
[2021-01-07 15:38:45,816] {hive_server_hook:162} INFO - Written 910000 rows so far.
[2021-01-07 15:38:46,275] {hive_server_hook:162} INFO - Written 920000 rows so far.
[2021-01-07 15:38:46,720] {hive_server_hook:162} INFO - Written 930000 rows so far.
[2021-01-07 15:38:47,116] {hive_server_hook:162} INFO - Written 940000 rows so far.
[2021-01-07 15:38:47,545] {hive_server_hook:162} INFO - Written 950000 rows so far.
[2021-01-07 15:38:47,970] {hive_server_hook:162} INFO - Written 960000 rows so far.
[2021-01-07 15:38:48,401] {hive_server_hook:162} INFO - Written 970000 rows so far.
[2021-01-07 15:38:48,824] {hive_server_hook:162} INFO - Written 980000 rows so far.
[2021-01-07 15:38:49,221] {hive_server_hook:162} INFO - Written 990000 rows 

[2021-01-07 15:39:25,804] {hive_server_hook:162} INFO - Written 1850000 rows so far.
[2021-01-07 15:39:26,230] {hive_server_hook:162} INFO - Written 1860000 rows so far.
[2021-01-07 15:39:26,664] {hive_server_hook:162} INFO - Written 1870000 rows so far.
[2021-01-07 15:39:27,062] {hive_server_hook:162} INFO - Written 1880000 rows so far.
[2021-01-07 15:39:27,492] {hive_server_hook:162} INFO - Written 1890000 rows so far.
[2021-01-07 15:39:27,917] {hive_server_hook:162} INFO - Written 1900000 rows so far.
[2021-01-07 15:39:28,345] {hive_server_hook:162} INFO - Written 1910000 rows so far.
[2021-01-07 15:39:28,758] {hive_server_hook:162} INFO - Written 1920000 rows so far.
[2021-01-07 15:39:29,219] {hive_server_hook:162} INFO - Written 1930000 rows so far.
[2021-01-07 15:39:29,647] {hive_server_hook:162} INFO - Written 1940000 rows so far.
[2021-01-07 15:39:30,070] {hive_server_hook:162} INFO - Written 1950000 rows so far.
[2021-01-07 15:39:30,514] {hive_server_hook:162} INFO - Written 1

[2021-01-07 15:40:07,074] {hive_server_hook:162} INFO - Written 2820000 rows so far.
[2021-01-07 15:40:07,146] {hive_server_hook:162} INFO - Written 2821741 rows so far.
[2021-01-07 15:40:07,225] {hiveserver2:265} INFO - Closing active operation
[2021-01-07 15:40:07,266] {hive_server_hook:163} INFO - Done. Loaded a total of 2821741 rows.
[2021-01-07 15:40:12,263] {<ipython-input-30-28c4b78419c8>:84} INFO - 读取数据完成.............


In [33]:
## 重命名
dl_site_ts_order_clean = dl_site_ts_order_clean.rename(columns={'license_year':'license_time_year','pay_year':'publish_time_year'})


In [34]:
## 剔除极大或者极小的峰值
dl_site_ts_order_clean_count = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).count().reset_index().rename(columns={'real_pay_amount':'count'})

dl_site_ts_order_clean_mean = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).mean().reset_index().rename(columns={'real_pay_amount':'mean'})

dl_site_ts_order_clean_std = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).std().reset_index().rename(columns={'real_pay_amount':'std'})

dl_site_ts_order_clean_median = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).median().reset_index().rename(columns={'real_pay_amount':'median'})

dl_site_ts_order_clean_min = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).min().reset_index().rename(columns={'real_pay_amount':'min_price'})

dl_site_ts_order_clean_max = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).max().reset_index().rename(columns={'real_pay_amount':'max_price'})

dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_count,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_mean,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_std,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_min,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_max,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_median,on=['model_code','license_time_year'],how='left')

## 剔除峰值
dl_site_ts_order_clean.loc[dl_site_ts_order_clean['mean']*0.3 < dl_site_ts_order_clean['std'],'std'] = dl_site_ts_order_clean['mean']*0.3

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median'] > dl_site_ts_order_clean['mean']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] < dl_site_ts_order_clean['mean']-dl_site_ts_order_clean['std']),'delete_flag'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median'] < dl_site_ts_order_clean['mean']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] > dl_site_ts_order_clean['mean']+dl_site_ts_order_clean['std']),'delete_flag'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median'] > dl_site_ts_order_clean['mean']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] < dl_site_ts_order_clean['mean']-dl_site_ts_order_clean['std']),'real_pay_amount'] = dl_site_ts_order_clean['mean']-dl_site_ts_order_clean['std']

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median'] < dl_site_ts_order_clean['mean']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] > dl_site_ts_order_clean['mean']+dl_site_ts_order_clean['std']),'real_pay_amount'] = dl_site_ts_order_clean['mean']+dl_site_ts_order_clean['std']



In [35]:
## 剔除极大或者极小的峰值
dl_site_ts_order_clean_count = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).count().reset_index().rename(columns={'real_pay_amount':'count_1'})

dl_site_ts_order_clean_mean = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).mean().reset_index().rename(columns={'real_pay_amount':'mean_1'})

dl_site_ts_order_clean_std = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).std().reset_index().rename(columns={'real_pay_amount':'std_1'})

dl_site_ts_order_clean_median = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).median().reset_index().rename(columns={'real_pay_amount':'median_1'})

dl_site_ts_order_clean_min = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).min().reset_index().rename(columns={'real_pay_amount':'min_price_1'})

dl_site_ts_order_clean_max = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).max().reset_index().rename(columns={'real_pay_amount':'max_price_1'})

dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_count,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_mean,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_std,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_min,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_max,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_median,on=['model_code','license_time_year'],how='left')

## 剔除峰值
dl_site_ts_order_clean.loc[dl_site_ts_order_clean['mean_1']*0.3 < dl_site_ts_order_clean['std_1'],'std_1'] = dl_site_ts_order_clean['mean_1']*0.3

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_1'] > dl_site_ts_order_clean['mean_1']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] < dl_site_ts_order_clean['mean_1']-dl_site_ts_order_clean['std_1']),'delete_flag_1'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_1'] < dl_site_ts_order_clean['mean_1']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] > dl_site_ts_order_clean['mean_1']+dl_site_ts_order_clean['std_1']),'delete_flag_1'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_1'] > dl_site_ts_order_clean['mean_1']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] < dl_site_ts_order_clean['mean_1']-dl_site_ts_order_clean['std_1']),'real_pay_amount'] = dl_site_ts_order_clean['mean_1']-dl_site_ts_order_clean['std_1']

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_1'] < dl_site_ts_order_clean['mean_1']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] > dl_site_ts_order_clean['mean_1']+dl_site_ts_order_clean['std_1']),'real_pay_amount'] = dl_site_ts_order_clean['mean_1']+dl_site_ts_order_clean['std_1']



In [36]:
## 剔除极大或者极小的峰值
dl_site_ts_order_clean_count = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).count().reset_index().rename(columns={'real_pay_amount':'count_2'})

dl_site_ts_order_clean_mean = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).mean().reset_index().rename(columns={'real_pay_amount':'mean_2'})

dl_site_ts_order_clean_std = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).std().reset_index().rename(columns={'real_pay_amount':'std_2'})

dl_site_ts_order_clean_median = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).median().reset_index().rename(columns={'real_pay_amount':'median_2'})

dl_site_ts_order_clean_min = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).min().reset_index().rename(columns={'real_pay_amount':'min_price_2'})

dl_site_ts_order_clean_max = dl_site_ts_order_clean[['model_code','license_time_year','real_pay_amount']].\
groupby(['model_code','license_time_year']).max().reset_index().rename(columns={'real_pay_amount':'max_price_2'})

dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_count,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_mean,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_std,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_min,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_max,on=['model_code','license_time_year'],how='left')
dl_site_ts_order_clean = pd.merge(dl_site_ts_order_clean,dl_site_ts_order_clean_median,on=['model_code','license_time_year'],how='left')

## 剔除峰值
dl_site_ts_order_clean.loc[dl_site_ts_order_clean['mean_2']*0.3 < dl_site_ts_order_clean['std_2'],'std_2'] = dl_site_ts_order_clean['mean_2']*0.3

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_2'] > dl_site_ts_order_clean['mean_2']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] < dl_site_ts_order_clean['mean_2']-dl_site_ts_order_clean['std_2']),'delete_flag_2'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_2'] < dl_site_ts_order_clean['mean_2']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] > dl_site_ts_order_clean['mean_2']+dl_site_ts_order_clean['std_2']),'delete_flag_2'] = 1

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_2'] > dl_site_ts_order_clean['mean_2']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] < dl_site_ts_order_clean['mean_2']-dl_site_ts_order_clean['std_2']),'real_pay_amount'] = dl_site_ts_order_clean['mean_2']-dl_site_ts_order_clean['std_2']

dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['median_2'] < dl_site_ts_order_clean['mean_2']) & 
#                           (dl_site_ts_order_clean['mean']*0.5 <= dl_site_ts_order_clean['std']) & 
                          (dl_site_ts_order_clean['real_pay_amount'] > dl_site_ts_order_clean['mean_2']+dl_site_ts_order_clean['std_2']),'real_pay_amount'] = dl_site_ts_order_clean['mean_2']+dl_site_ts_order_clean['std_2']




In [40]:
## 保存数据
dl_site_ts_order_clean[['car_id','real_pay_amount','delete_flag','delete_flag_1','delete_flag_2']].to_csv(curr_dir+"dl_site_ts_order_clean_"+computer_year+"_yh01.csv")


In [38]:
print(dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['delete_flag'] == 1)].shape)
print(dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['delete_flag_1'] == 1)].shape)
print(dl_site_ts_order_clean.loc[(dl_site_ts_order_clean['delete_flag_2'] == 1)].shape)



(354968, 32)
(454127, 32)
(541769, 32)
